In [7]:
!pip install imutils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
#!pip3 install opencv-contrib-python

In [4]:
#!conda remove opencv
#!conda install -c conda-forge opencv=4.1.0

In [8]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from sklearn.preprocessing import LabelBinarizer
from keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from keras.applications import VGG19
from imutils import paths
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pickle

ImportError: ignored

In [ ]:
print(tensorflow.__version__)

In [ ]:
dataset = r"F:/data_science/Blu-Cocoon/Plant_Disease_Detection/Large Wheat Disease Classification Dataset/Wheat-Disease-Types/Train"
#label = r"E:\Wheat Disease Detection\lb.pickle"

In [ ]:
LABELS = set(["Crown and Root Rot", "Healthy Wheat", "Leaf Rust", "Wheat Loose Smut"])

In [ ]:
imagePaths = list(paths.list_images(dataset))
data = []
labels = []

In [ ]:
len(imagePaths)

In [ ]:
# loop over the image paths
for imagePath in imagePaths:
 # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    if label not in LABELS:
        continue
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

In [ ]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
 test_size=0.25, stratify=labels, random_state=42)

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
 rotation_range=30,
 zoom_range=0.15,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.15,
 horizontal_flip=True,
 fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [ ]:
'''# load the VGG19 network, ensuring the head FC layer sets are left
# off
headmodel = VGG19(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))
#headmodel = VGG19(weights="imagenet", include_top=False,
                  #input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
model = headmodel.output
model = AveragePooling2D(pool_size=(5, 5))(model)
model = Flatten(name="flatten")(model)
model = Dense(512, activation="relu")(model)
model = Dropout(0.4)(model)
model = Dense(len(lb.classes_), activation="softmax")(model)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
moodel = Model(inputs=headmodel.input, outputs=model)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in headmodel.layers:
    layer.trainable = False'''

In [ ]:
headmodel = tensorflow.keras.applications.ResNet50(include_top=True,weights="imagenet",input_tensor=Input(shape=(224, 224, 3))
                                           )
# load the VGG19 network, ensuring the head FC layer sets are left
# off
#headmodel = VGG19(weights="imagenet", include_top=False,
                  #input_tensor=Input(shape=(224, 224, 3)))
#headmodel = VGG19(weights="imagenet", include_top=False,
                  #input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
model = headmodel.output
#model = AveragePooling2D(pool_size=(5, 5))(model)
model = Flatten(name="flatten")(model)
model = Dense(512, activation="relu")(model)
model = Dropout(0.4)(model)
model = Dense(len(lb.classes_), activation="softmax")(model)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
moodel = Model(inputs=headmodel.input, outputs=model)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in headmodel.layers:
    layer.trainable = False

In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
opt = Adam(learning_rate=1e-3)
#opt = Adam(Learning_rate=0.001)
moodel.compile(loss="categorical_crossentropy", optimizer=opt,
               metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
H = moodel.fit(
    trainAug.flow(trainX, trainY, batch_size=32),
    steps_per_epoch=len(trainX) // 32,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // 32,
    epochs=100)

In [ ]:
# evaluate the network
predictions = moodel.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = 100
plt.plot(np.arange(0, N), H.history['accuracy'], label="Training Accuracy")
plt.plot(np.arange(0, N), H.history['val_accuracy'], label="Test Accuracy")
plt.title('VGG19 Model Train vs Test Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()
#plt.savefig(r"E:\Wheat Disease Detection\Accuracy_Plot.png")

plt.plot(H.history['loss'], label="Training Loss")
plt.plot(H.history['val_loss'], label="Test Loss")
plt.title('VGG19 Model Train vs Test Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()
#plt.savefig(r"E:\Wheat Disease Detection\Loss_Plot.png")

In [ ]:
#Save the Model and label file to Disk
tf.keras.moodel.save("Final_wheat_detection.pb")
f = open("label", "wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
model_path = "saved_model.pb"
input = "Add test image path here"
imagePaths_test = list(paths.list_images(input))
label = "label"
# load the trained model and label binarizer from disk
moodel = load_model(model_path)
lb = pickle.loads(open("label", "rb").read())
# initialize the image mean for mean subtraction along with the
# predictions queue
for path_i in imagePaths_test:
    mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
    Q = deque(maxlen=128)
    vs = cv2.VideoCapture(path_i)
    (W, H) = (None, None)
    while True:
        (grabbed, frame) = vs.read()
        if not grabbed:
            break
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224)).astype("float32")
        frame -= mean

        preds = moodel.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        results = np.array(Q).mean(axis=0)
        i = np.argmax(results)
        label = lb.classes_[i]

        text = "PREDICTION: {}".format(label.upper())
        cv2.putText(output, text, (4, 4), cv2.FONT_HERSHEY_SIMPLEX,0.25, (200,255,155), 2)

        # show the output image
        #print(output.shape)
        print(text)
        plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        plt.show()
        #cv2.imshow("Output",output)
        #key = cv2.waitKey(10) & 0xFF

        # if the `q` key was pressed, break from the loop
        #if key == ord("q"):
            #break

        
vs.release()

In [ ]:
imagePaths_test

In [ ]:
label

In [ ]:
pwd

In [ ]:
ls